In [1]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OrdinalEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from pathlib import Path

C:\Users\Vasco\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\scipy\__init__.py:177: UserWarning: A NumPy version >=1.18.5 and <1.26.0 is required for this version of SciPy (detected version 1.26.1
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
def train(df: pd.DataFrame) :
    # regression model
    rf = RandomForestRegressor(n_estimators=100, random_state=42)



    # TODO: remove date and add year, quarter, maybe wd_quarter (wd within quarter)

    df['brand'] = df['brand'].astype('category')
    df['country'] = df['country'].astype('category')
    df['dayweek'] = df['dayweek'].astype('category')
    df['month'] = df['month'].astype('category')
    df['wd'] = df['wd'].astype('category')
    df['ther_area'] = np.where(df['ther_area'].isnull(), 'unknown', df['ther_area'])
    df['main_channel'] = np.where(df['main_channel'].isnull(), 'unknown', df['main_channel'])


    # categoric_col = ['ther_area', 'main_channel','dayweek','month','wd']
    ordinal_col = ['dayweek', 'month', 'wd']

    #pre_processor = ColumnTransformer(transformers=[('cat', OneHotEncoder(handle_unknown='ignore'), categoric_col)], remainder='passthrough')
    pre_processor = ColumnTransformer(transformers=[('ord', OrdinalEncoder(), ordinal_col)], remainder='passthrough')
    pipeline = Pipeline(steps=[('preprocessor', pre_processor), ('regressor', rf)])


    # missing hospital rate

    #x_values = df[[ 'dayweek', 'month', 'wd', 'ther_area','hospital_rate', 'main_channel', 'monthly', 'n_nwd_bef', 'n_nwd_aft']]
    x_values = df[[ 'dayweek', 'month', 'wd', 'monthly', 'n_nwd_bef', 'n_nwd_aft']]
    y_values = df['phase']

    #x_train, x_test, y_train, y_test = train_test_split(x_values, y_values, test_size=0.0, random_state=42)

    print("\nstarted fitting\n")
    rf = pipeline.fit(x_values, y_values)
    print("\nfinished fitting\n")

    return rf



In [3]:
PATH = Path("../data")
train_data = pd.read_parquet(PATH / "train_data.parquet")

In [4]:
model = train(train_data)



started fitting


finished fitting



In [12]:
submission_data = pd.read_parquet(PATH / "submission_data.parquet")
submission = pd.read_csv(PATH / "submission_template.csv")

# Fill in 'prediction' values of submission




In [13]:
submission_data['monthly'] = train_data['monthly']
submission_data['brand'] = submission_data['brand'].astype('category')
submission_data['country'] = submission_data['country'].astype('category')
submission_data['dayweek'] = submission_data['dayweek'].astype('category')
submission_data['month'] = submission_data['month'].astype('category')
submission_data['wd'] = submission_data['wd'].astype('category')
submission_data['ther_area'] = np.where(submission_data['ther_area'].isnull(), 'unknown', submission_data['ther_area'])
submission_data['main_channel'] = np.where(submission_data['main_channel'].isnull(), 'unknown', submission_data['main_channel'])

submission_data['monthly'].fillna(submission_data['monthly'].mean(), inplace=True)
submission_data = submission_data[['brand','country','date','dayweek', 'month', 'wd', 'monthly', 'n_nwd_bef', 'n_nwd_aft']]

submission['prediction'] = model.predict(submission_data)

# ...

# Save submission
SAVE_PATH = Path("../data")
submission.to_csv(SAVE_PATH / "submissionv1.csv", index=False)

brand            0
country          0
date             0
dayweek          0
month            0
wd               0
monthly      35215
n_nwd_bef        0
n_nwd_aft        0
dtype: int64


ValueError: Input X contains NaN.
RandomForestRegressor does not accept missing values encoded as NaN natively. For supervised learning, you might want to consider sklearn.ensemble.HistGradientBoostingClassifier and Regressor which accept missing values encoded as NaNs natively. Alternatively, it is possible to preprocess the data, for instance by using an imputer transformer in a pipeline or drop samples with missing values. See https://scikit-learn.org/stable/modules/impute.html You can find a list of all estimators that handle NaN values at the following page: https://scikit-learn.org/stable/modules/impute.html#estimators-that-handle-nan-values